![](images/EscUpmPolit_p.gif "UPM")

# Telco Churn

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import random
import numpy as np
import pandas as pd
from sklearn import datasets, svm, cross_validation, tree, preprocessing, metrics
import sklearn.ensemble as ske
import seaborn as sns
sns.set(color_codes=True)
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [ ]:
df = pd.read_csv('data-churn/train.csv')
df[:5]

In [ ]:
df.count()

Se comprueba si tenemos algún elemento nulo en el dataset.

In [ ]:
df.isnull().sum()

Se comprueba el tipo de datos que contiene el dataset

In [ ]:
df.info()

Se filtran los elementos que no son de tipo numérico, es decir aquellos que no son ni int64 ni float64. Estos serán los elementos que se codificarán de manera continua.

In [ ]:
df.dtypes[df.dtypes == object]

Vemos los valores de los elementos anteriores de tipo object.

In [ ]:
print("gender: "+df['gender'].unique())
print("Parther: "+df['Partner'].unique())
print("Dependents: "+df['Dependents'].unique())
print("PhoneService: "+df['PhoneService'].unique())
print("MultipleLines: "+df['MultipleLines'].unique())
print("InternetService: "+df['InternetService'].unique())
print("OnlineSecurity: "+df['OnlineSecurity'].unique())
print("OnlineBackup: "+df['OnlineBackup'].unique())
print("DeviceProtection: "+df['DeviceProtection'].unique())
print("TechSupport: "+df['TechSupport'].unique())
print("StreamingTV: "+df['StreamingTV'].unique())
print("StreamingMovies: "+df['StreamingMovies'].unique())
print("Contract: "+df['Contract'].unique())
print("PaperlessBilling: "+df['PaperlessBilling'].unique())
print("PaymentMethod: "+df['PaymentMethod'].unique())
print("TotalCharges: "+df['TotalCharges'].unique())

Lo siguiente será comprobar si existen duplicados. En este caso comprobamos que no tenemos información duplicada.

In [ ]:
df.duplicated().any()

A continuación se pasa a codificar los valores como variables continuas.

In [ ]:
df.loc[df["gender"] == "Male", "gender"] = 0
df.loc[df["gender"] == "Female", "gender"] = 1

df.loc[df["Partner"] == "Yes", "Partner"] = 1
df.loc[df["Partner"] == "No", "Partner"] = 0

df.loc[df["Dependents"] == "Yes", "Dependents"] = 1
df.loc[df["Dependents"] == "No", "Dependents"] = 0

df.loc[df["PhoneService"] == "Yes", "PhoneService"] = 1
df.loc[df["PhoneService"] == "No", "PhoneService"] = 0

df.loc[df["MultipleLines"] == "No", "MultipleLines"] = 0
df.loc[df["MultipleLines"] == "No phone service", "MultipleLines"] = 0
df.loc[df["MultipleLines"] == "Yes", "MultipleLines"] = 1

df.loc[df["InternetService"] == "No", "InternetService"] = 0
df.loc[df["InternetService"] == "DSL", "InternetService"] = 1
df.loc[df["InternetService"] == "Fiber optic", "InternetService"] = 2

df.loc[df["OnlineSecurity"] == "No", "OnlineSecurity"] = 0
df.loc[df["OnlineSecurity"] == "No internet service", "OnlineSecurity"] = 0
df.loc[df["OnlineSecurity"] == "Yes", "OnlineSecurity"] = 1

df.loc[df["OnlineBackup"] == "No", "OnlineBackup"] = 0
df.loc[df["OnlineBackup"] == "No internet service", "OnlineBackup"] = 0
df.loc[df["OnlineBackup"] == "Yes", "OnlineBackup"] = 1

df.loc[df["DeviceProtection"] == "No", "DeviceProtection"] = 0
df.loc[df["DeviceProtection"] == "No internet service", "DeviceProtection"] = 0
df.loc[df["DeviceProtection"] == "Yes", "DeviceProtection"] = 1

df.loc[df["TechSupport"] == "No", "TechSupport"] = 0
df.loc[df["TechSupport"] == "No internet service", "TechSupport"] = 0
df.loc[df["TechSupport"] == "Yes", "TechSupport"] = 1

df.loc[df["StreamingTV"] == "No", "StreamingTV"] = 0
df.loc[df["StreamingTV"] == "No internet service", "StreamingTV"] = 0
df.loc[df["StreamingTV"] == "Yes", "StreamingTV"] = 1

df.loc[df["StreamingMovies"] == "No", "StreamingMovies"] = 0
df.loc[df["StreamingMovies"] == "No internet service", "StreamingMovies"] = 0
df.loc[df["StreamingMovies"] == "Yes", "StreamingMovies"] = 1

df.loc[df["Contract"] == "Two year", "Contract"] = 0
df.loc[df["Contract"] == "One year", "Contract"] = 1
df.loc[df["Contract"] == "Month-to-month", "Contract"] = 2

df.loc[df["PaperlessBilling"] == "Yes", "PaperlessBilling"] = 1
df.loc[df["PaperlessBilling"] == "No", "PaperlessBilling"] = 0

df.loc[df["PaymentMethod"] == "Bank transfer (automatic)", "PaymentMethod"] = 0
df.loc[df["PaymentMethod"] == "Mailed check", "PaymentMethod"] = 1
df.loc[df["PaymentMethod"] == "Electronic check", "PaymentMethod"] = 2
df.loc[df["PaymentMethod"] == "Credit card (automatic)", "PaymentMethod"] = 3

df["TotalCharges"] = df["TotalCharges"].apply(pd.to_numeric, errors='coerce')

df[-5:]

Tras realizar las modificaciones se vuelve a comprobar que no se tenga ningún elemento Null

In [ ]:
df.isnull().sum()

Se comprueba que la columna TotalCharges contiene 10 valores nulos. Se sustituyen por la media.

In [ ]:
df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].median())

In [ ]:
df.isnull().sum()

Se vuelve a comprobar que no haya duplicados.

In [ ]:
df.duplicated().any()

Ahora separamos los datos en dos arrays: Uno X que contendra todos los valores de cada fila menos los de churn. y otro Y que contendrá solo los datos de churn para esa fila. El algoritmo de clasificacion comparara los valores de atributo de X a sus correspondientes valores de Y para detectar patrones de como diferentes valores de atributo afectan a la permanencia de un cliente.

In [ ]:
X = df.drop(['Churn'], axis=1).values
y = df['Churn'].values

Ahora dividimos los arrays X e Y en dos partes, un set de entrenamiento y otro de testeo. Daremos el set de entrenamiento al algoritmo de clasificación para crear un modelo entrenado. Una vez el modelo se haya formado, lo utilizaremos para clasificar el set de testeo, lo que nos permitira determinar la precision del modelo. Aquí he hecho un reparto de 20-80, de tal manera que el 80% del dataset se utilizará para entrenamiento y el 20% para testeo:

In [ ]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,y,test_size=0.2)

Primero comenzaremos con un arbol de decision (Tecnica aprendida en clase.) El arbol de decision examinara una variable cada vez, y se dividira entre 1 de 2 ramas dependiendo del resultado del valor obtenido. 
De tal manera que el árbol primero se divide por sexo, y después por clase, ya que habrá aprendido en la secuencia de entrenamiento que estas son las dos principales caracteristicas que determinan la suerte de un individuo. 

In [ ]:
clf_dt = tree.DecisionTreeClassifier(max_depth=10)

In [ ]:
clf_dt.fit (X_train, y_train)
clf_dt.score (X_test, y_test)

Como sabemos, la precisión del modelo puede variar dependiendo de que filas sean seleccionadas para las secuencias de entrenamiento y testeo. Para que esto no condicione los resultados vamos a poner un factor de aleatoriedad:

In [ ]:
shuffle_validator = cross_validation.ShuffleSplit(len(X), n_iter=20, test_size=0.2, random_state=0)
def test_classifier(clf):
    scores = cross_validation.cross_val_score(clf, X, y, cv=shuffle_validator)
    print("Accuracy: %0.4f (+/- %0.2f)" % (scores.mean(), scores.std()))

In [ ]:
test_classifier(clf_dt)

In [ ]:
y_true, y_pred = y_test, clf_dt.predict(X_test)
print(classification_report(y_true, y_pred))
print (f1_score(y_true, y_pred))

In [ ]:
print("Prediction ", clf_dt.predict(X))

Se intentan optimizar los parámetros empleando Grid Search

In [ ]:
# set of parameters to test
tuned_parameters = [{'max_depth': np.arange(3, 10),
                     'criterion': ['gini', 'entropy'], 
                     'splitter': ['best', 'random'],
                     'min_samples_leaf': [2, 5, 10],
                     'class_weight':['balanced'],
                     'max_leaf_nodes': [None, 5, 10, 20]
                    }]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    # cv = the fold of the cross-validation cv, defaulted to 5
    gs = GridSearchCV(tree.DecisionTreeClassifier(), tuned_parameters, cv=10, scoring='%s_weighted' % score)
    gs.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(gs.best_params_)
    print()
    print("Detailed classification report:")
    print()

    y_true, y_pred = y_test, gs.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

Se modifica el modelo usando los parámetros optimizados obtenidos previamente.

In [ ]:
clf_dt = tree.DecisionTreeClassifier(class_weight='balanced', criterion='gini',max_depth=8,
                                    max_leaf_nodes=20,min_samples_leaf=2,splitter='random')

In [ ]:
clf_dt.fit (X_train, y_train)
clf_dt.score (X_test, y_test)

In [ ]:
shuffle_validator = cross_validation.ShuffleSplit(len(X), n_iter=20, test_size=0.2, random_state=0)
def test_classifier(clf):
    scores = cross_validation.cross_val_score(clf, X, y, cv=shuffle_validator)
    print("Accuracy: %0.4f (+/- %0.2f)" % (scores.mean(), scores.std()))

In [ ]:
test_classifier(clf_dt)

In [ ]:
print(classification_report(y_true, y_pred))
print (f1_score(y_true, y_pred))

In [ ]:
dfTest = pd.read_csv('telco-churn/test_nolabel.csv')

In [ ]:
dfTest.loc[dfTest["gender"] == "Male", "gender"] = 0
dfTest.loc[dfTest["gender"] == "Female", "gender"] = 1

dfTest.loc[dfTest["Partner"] == "Yes", "Partner"] = 1
dfTest.loc[dfTest["Partner"] == "No", "Partner"] = 0

dfTest.loc[dfTest["Dependents"] == "Yes", "Dependents"] = 1
dfTest.loc[dfTest["Dependents"] == "No", "Dependents"] = 0

dfTest.loc[dfTest["PhoneService"] == "Yes", "PhoneService"] = 1
dfTest.loc[dfTest["PhoneService"] == "No", "PhoneService"] = 0

dfTest.loc[dfTest["MultipleLines"] == "No", "MultipleLines"] = 0
dfTest.loc[dfTest["MultipleLines"] == "No phone service", "MultipleLines"] = 0
dfTest.loc[dfTest["MultipleLines"] == "Yes", "MultipleLines"] = 1

dfTest.loc[dfTest["InternetService"] == "No", "InternetService"] = 0
dfTest.loc[dfTest["InternetService"] == "DSL", "InternetService"] = 1
dfTest.loc[dfTest["InternetService"] == "Fiber optic", "InternetService"] = 2

dfTest.loc[dfTest["OnlineSecurity"] == "No", "OnlineSecurity"] = 0
dfTest.loc[dfTest["OnlineSecurity"] == "No internet service", "OnlineSecurity"] = 0
dfTest.loc[dfTest["OnlineSecurity"] == "Yes", "OnlineSecurity"] = 1

dfTest.loc[dfTest["OnlineBackup"] == "No", "OnlineBackup"] = 0
dfTest.loc[dfTest["OnlineBackup"] == "No internet service", "OnlineBackup"] = 0
dfTest.loc[dfTest["OnlineBackup"] == "Yes", "OnlineBackup"] = 1

dfTest.loc[dfTest["DeviceProtection"] == "No", "DeviceProtection"] = 0
dfTest.loc[dfTest["DeviceProtection"] == "No internet service", "DeviceProtection"] = 0
dfTest.loc[dfTest["DeviceProtection"] == "Yes", "DeviceProtection"] = 1

dfTest.loc[dfTest["TechSupport"] == "No", "TechSupport"] = 0
dfTest.loc[dfTest["TechSupport"] == "No internet service", "TechSupport"] = 0
dfTest.loc[dfTest["TechSupport"] == "Yes", "TechSupport"] = 1

dfTest.loc[dfTest["StreamingTV"] == "No", "StreamingTV"] = 0
dfTest.loc[dfTest["StreamingTV"] == "No internet service", "StreamingTV"] = 0
dfTest.loc[dfTest["StreamingTV"] == "Yes", "StreamingTV"] = 1

dfTest.loc[dfTest["StreamingMovies"] == "No", "StreamingMovies"] = 0
dfTest.loc[dfTest["StreamingMovies"] == "No internet service", "StreamingMovies"] = 0
dfTest.loc[dfTest["StreamingMovies"] == "Yes", "StreamingMovies"] = 1

dfTest.loc[dfTest["Contract"] == "Two year", "Contract"] = 0
dfTest.loc[dfTest["Contract"] == "One year", "Contract"] = 1
dfTest.loc[dfTest["Contract"] == "Month-to-month", "Contract"] = 2

dfTest.loc[dfTest["PaperlessBilling"] == "Yes", "PaperlessBilling"] = 1
dfTest.loc[dfTest["PaperlessBilling"] == "No", "PaperlessBilling"] = 0

dfTest.loc[dfTest["PaymentMethod"] == "Bank transfer (automatic)", "PaymentMethod"] = 0
dfTest.loc[dfTest["PaymentMethod"] == "Mailed check", "PaymentMethod"] = 1
dfTest.loc[dfTest["PaymentMethod"] == "Electronic check", "PaymentMethod"] = 2
dfTest.loc[dfTest["PaymentMethod"] == "Credit card (automatic)", "PaymentMethod"] = 3

dfTest["TotalCharges"] = dfTest["TotalCharges"].apply(pd.to_numeric, errors='coerce')
dfTest['TotalCharges'] = dfTest['TotalCharges'].fillna(dfTest['TotalCharges'].median())
dfTest[-5:]


In [ ]:
X2 = dfTest

In [ ]:
churn = clf_dt.predict(X2)

In [ ]:
dataframeResult = dfTest.filter(['customerID'], axis=1)

In [ ]:
dataframeResult['Churn'] = churn

In [ ]:
dataframeResult[:]

In [ ]:
dataframeResult.to_csv('out.csv', sep='\t')